# **MODULE 2: Model Tuning**




#### At the end of this module participants should be able to: 
           1. Understand the theory behind parameter space explorations
           2. Select appropriate model parameter value ranges
           3. Run a parameter space exploration
           4. Weigh practical considerations when tuning a model
           
A significant portion of our workflow is adapted from the [Virtual Aging Brain GitHub Repository](https://github.com/ins-amu/virtual_aging_brain) and the [Virtual Ageing Showcase on EBRAINS](https://wiki.ebrains.eu/bin/view/Collabs/sga3-d1-2-showcase-1/).

Sample data was obtained from the [Amsterdam PIOP2 Open Dataset](https://nilab-uva.github.io/AOMIC.github.io/) and prepared for simulation using the [TVB-UKBB MRI Processing Pipeline](https://github.com/McIntosh-Lab/tvb-ukbb).

<br>

---


### *1. Introduction to Parameter Space Exploration (PSE)*

A **Parameter Space Exploration (PSE)** evaluates how a model behaves across different parameter values. By running the model many times with different parameter combinations, we can identify which settings produce meaningful and realistic brain network features.

In this module, we use PSE to find parameter values that **maximize functional connectivity dynamics (FCD) variance**, helping to identify model configurations that produce switching dynamics in the brain network activity.


<br>

Below is an example output from a PSE showing how FCD variance changes across the model parameter space. Blue regions indicate higher FCD variance, yellow regions indicate lower FCD variance, and red regions represent parameter combinations that produced non-valid or unstable model outputs (NaN). 

![image.png](./_img/PSE_image.png)


![image.png](./_img/PSE_things_to_modify.png)


### *2. Load in the required packages for this notebook*

In [ ]:
%pylab inline
import sys, os, time
import numpy as np
import src
from src import viz
from src import simulation
from src import analysis
from tvb.simulator.lab import *
from tvb.simulator.backend.nb_mpr import NbMPRBackend
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.signal import savgol_filter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patheffects as pe
from matplotlib.colors import ListedColormap

### *3. Visualizing the effects of model parameters and simulation settings*

Using the connectome of `sub-0001`, a example subject from the Amsterdam Dataset, let's investigate how each model parameter and simulation setting affects our simulated outputs. 

Let's first define the function to load in connectomes.

In [ ]:
# Define the function to Load in the SC file - function to be called later
def get_connectivity(scaling_factor,sc_path):
        SC = np.loadtxt(sc_path)
        SC = SC / scaling_factor
        conn = connectivity.Connectivity(
                weights = SC,
                tract_lengths=np.ones_like(SC),
                centres = np.zeros(np.shape(SC)[0]),
                speed = np.r_[np.Inf]
        )
        conn.compute_region_labels()

        return conn
    
sub_dir='/tvb_node/tvb/tvb-node-mclab/Session_Materials/data/sub-0001/' #MODIFY
scaling_factor=1     #scaling the SC matrix - strength of connections not changed when 1 - no normalization
sc_path = os.path.join(sub_dir,'weights.txt')

<br>

#### Simulation Core - **Baseline**
The following segment of code will run a simulation. This will serve as a baseline simulation with parameter settings: `G=1.993, nsigma=0.04, dt=0.005, sim_len=4e3`. When we modify our parameter values later, we can compare the simulation outputs with this baseline.



In [ ]:
# Specify the model and model initial parameter values
G=1.99      #global coupling
nsigma=0.045  #noise variance
dt=0.01    #integration step size

sim_len=10e3   #length of neural activity to be simulated. With the current setup, 2e3 is 20 seconds.

# Set up a simulation object
sim = simulator.Simulator(
    connectivity = get_connectivity(scaling_factor,sc_path),
    model = models.MontbrioPazoRoxin(
        eta   = np.r_[-4.6],
        J     = np.r_[14.5],
        Delta = np.r_[0.7],
        tau   = np.r_[1.],
    ),
    coupling = coupling.Linear(a=np.r_[G]),
    integrator = integrators.HeunStochastic(
        dt = dt,
        noise = noise.Additive(nsig=np.r_[nsigma, nsigma*2], noise_seed=2)
    ),
    monitors = [monitors.TemporalAverage(period=0.1)]
).configure()

# Run the simulation
runner = NbMPRBackend()

start_time = time.time() #mark start time 

(tavg_t, tavg_d), = runner.run_sim(sim, simulation_length=sim_len)   #run the sim

end_time = time.time() #mark end time
elapsed_time = end_time - start_time
print(f"Simulation took: {elapsed_time} seconds")

tavg_t *= 10 #convert simulation timepoints to ms


#Apply the Windkessel model to the simulated data to derive the BOLD time series with TR=2000ms
bold_t, bold_d = simulation.tavg_to_bold(tavg_t, tavg_d, tavg_period=1, connectivity=sim.connectivity, svar=0, decimate=2000) 

# Cut the initial transient (e.g., 16 seconds). First ~15 seconds of the balloon model output should be discarded 
bold_t = bold_t[8:] 
bold_d = bold_d[8:]


<br> 

#### Visualizing Simulated Activity - **Baseline**
The following cell visualizes the simulated activity of the first 10 brain regions. 

The cell will also create a regional activity trajectory plot, visualizing the simulated membrane potential (v) and firing rate (r) of a single brain region over 10 seconds. This may give you some insight into the bistability of a region given your model parameters, but it _should not be taken as a reliable description of bistability for this brain region_.



In [ ]:
ROIs_to_plot = range(10)

r_min_found, r_max_found = 0.0, 3.52075
v_min_found, v_max_found = -4.10274, 3.94877

r_pad = 0.1
v_pad = 0.2

r_lim = (r_min_found - r_pad, r_max_found + r_pad)   # (-0.1, 3.62075)
v_lim = (v_min_found - v_pad, v_max_found + v_pad)   # (-4.30274, 4.14877)

fig = plt.figure(figsize=(16, 13), constrained_layout=True)

gs = fig.add_gridspec(
    nrows=3, ncols=2,
    height_ratios=[1.0, 2.2, 0.55],
    width_ratios=[1.0, 1.0]
)


# 1) Simulated firing rate plot
ax1 = fig.add_subplot(gs[0, :])

r_min_found, r_max_found = 0.0, 3.52075
r_pad = 0.1
r_lim = (r_min_found - r_pad, r_max_found + r_pad)   

fr_data = tavg_d[15000:, 0, ROIs_to_plot, 0]
fr_data = np.clip(fr_data, r_lim[0], r_lim[1])

ax1 = viz.plot_ts_stack(
    fr_data,
    x=tavg_t[15000:] / 1000,
    width=20,
    ax=ax1
)

ax1.set(xlabel="time [s]")
ax1.set(ylabel="ROI")
ax1.set_title("Simulated firing rate")

ax1.set_yticks(list(ROIs_to_plot))
ax1.set_yticklabels([f"{t+1:.0f}" for t in ROIs_to_plot])



# 2) regional activity trajectory plot
ax2 = fig.add_subplot(gs[1, 0])

roi = 6
start = 20000
end = 25000

r = tavg_d[start:end, 0, roi, 0]
v = tavg_d[start:end, 1, roi, 0]

print(f"[ROI {roi}] RAW r: min={np.min(r):.6g}, max={np.max(r):.6g}")
print(f"[ROI {roi}] RAW v: min={np.min(v):.6g}, max={np.max(v):.6g}")

r_vect = savgol_filter(r, 15, 3)
v_vect = savgol_filter(v, 15, 3)
data_rv = pd.DataFrame({"r": r_vect, "v": v_vect})

sns.set_theme(style="darkgrid")
sns.lineplot(data=data_rv, x="r", y="v", sort=False, lw=2, color="blue", ax=ax2)
ax2.set_xlabel("Firing Rate", fontsize=12)
ax2.set_ylabel("Membrane Potential", fontsize=12)
ax2.set_title("Regional Activity Trajectory Plot (ROI 6)")

ax2.set_xlim((r_min_found - r_pad, 1.8 + r_pad))
ax2.set_ylim((-2.5 - v_pad, 2 + v_pad))

ax2.set_box_aspect(1)



# 3) FCD matrix
ax3 = fig.add_subplot(gs[1, 1])

FCD, _, _ = analysis.compute_fcd(bold_d[:, 0, :, 0], win_len=5)

im = ax3.imshow(FCD, interpolation="nearest")
ax3.set_title("FCD Plot")
ax3.set_xlabel("Timepoint")
ax3.set_ylabel("Timepoint")
ax3.grid(False)

ax3.set_box_aspect(1)
ax3.set_aspect("equal")

cax = inset_axes(ax3, width="4%", height="85%", loc="right", borderpad=1)
plt.colorbar(im, cax=cax, label="Correlation")



# 4) text summary
ax4 = fig.add_subplot(gs[2, :])
ax4.axis("off")
fcd_var = np.var(np.triu(FCD, k=5))
summary_text = (
    f"FCD Variance: {fcd_var:.6e}\n"
)
ax4.text(
    0.5, 0.5, summary_text,
    fontsize=12, ha="center", va="center",
    bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5)
)



# Save img
fig.savefig(f"Module-2_output_images/BASELINE_G-{G}_noise-{nsigma}_dt-{dt}_simlen{sim_len}_output-figs.png", dpi=60, bbox_inches="tight")
plt.show()




<br>

Note that, with short simulations (40s), the FCD matrix shows little recurrence of functional networks over time. This is also reflected in low FCD variance.

<br>

---

<br>

### Exploring Model Parameters

Let's see how our simulations change as model parameters and simulation settings vary. Note changes in the simulated time series, biphase plot, and functional dynamics.

<br>

#### Simulation Core - **Exploratory**

The following code is the same as the `Simulation Core - Baseline` above. Take some time to rerun multiple simulations below, making sure to modify the model parameters and simulation settings: `G`, `nsigma`, and `sim_len`. 

In [ ]:
# Specify the model and model initial parameter values
G=1.99      #global coupling
nsigma=0.045  #noise variance
dt=0.01    #integration step size

sim_len=10e3   #length of neural activity to be simulated. With the current setup, 2e3 is 20 seconds.

# Set up a simulation object
sim = simulator.Simulator(
    connectivity = get_connectivity(scaling_factor,sc_path),
    model = models.MontbrioPazoRoxin(
        eta   = np.r_[-4.6],
        J     = np.r_[14.5],
        Delta = np.r_[0.7],
        tau   = np.r_[1.],
    ),
    coupling = coupling.Linear(a=np.r_[G]),
    integrator = integrators.HeunStochastic(
        dt = dt,
        noise = noise.Additive(nsig=np.r_[nsigma, nsigma*2], noise_seed=2)
    ),
    monitors = [monitors.TemporalAverage(period=0.1)]
).configure()

# Run the simulation
runner = NbMPRBackend()

start_time = time.time() #mark start time 

(tavg_t, tavg_d), = runner.run_sim(sim, simulation_length=sim_len)   #run the sim

end_time = time.time() #mark end time
elapsed_time = end_time - start_time
print(f"Simulation took: {elapsed_time} seconds")

tavg_t *= 10 #convert simulation timepoints to ms


#Apply the Windkessel model to the simulated data to derive the BOLD time series with TR=2000ms
bold_t, bold_d = simulation.tavg_to_bold(tavg_t, tavg_d, tavg_period=1, connectivity=sim.connectivity, svar=0, decimate=2000) 

# Cut the initial transient (e.g., 16 seconds). First ~15 seconds of the balloon model output should be discarded 
bold_t = bold_t[8:] 
bold_d = bold_d[8:]

<br> 

#### Visualization - **Exploratory**

The below cell will output visualizations and FCD variance for your modified simulation.

In [ ]:
ROIs_to_plot = range(10)

r_min_found, r_max_found = 0.0, 3.52075
v_min_found, v_max_found = -4.10274, 3.94877

r_pad = 0.1
v_pad = 0.2

r_lim = (r_min_found - r_pad, r_max_found + r_pad)   # (-0.1, 3.62075)
v_lim = (v_min_found - v_pad, v_max_found + v_pad)   # (-4.30274, 4.14877)

fig = plt.figure(figsize=(16, 13), constrained_layout=True)

gs = fig.add_gridspec(
    nrows=3, ncols=2,
    height_ratios=[1.0, 2.2, 0.55],
    width_ratios=[1.0, 1.0]
)


# 1) Simulated firing rate plot
ax1 = fig.add_subplot(gs[0, :])

r_min_found, r_max_found = 0.0, 3.52075
r_pad = 0.1
r_lim = (r_min_found - r_pad, r_max_found + r_pad)   

fr_data = tavg_d[15000:, 0, ROIs_to_plot, 0]
fr_data = np.clip(fr_data, r_lim[0], r_lim[1])

ax1 = viz.plot_ts_stack(
    fr_data,
    x=tavg_t[15000:] / 1000,
    width=20,
    ax=ax1
)

ax1.set(xlabel="time [s]")
ax1.set(ylabel="ROI")
ax1.set_title("Simulated firing rate")

ax1.set_yticks(list(ROIs_to_plot))
ax1.set_yticklabels([f"{t+1:.0f}" for t in ROIs_to_plot])



# 2) regional activity trajectory plot
ax2 = fig.add_subplot(gs[1, 0])

roi = 6
start = 20000
end = 25000

r = tavg_d[start:end, 0, roi, 0]
v = tavg_d[start:end, 1, roi, 0]

print(f"[ROI {roi}] RAW r: min={np.min(r):.6g}, max={np.max(r):.6g}")
print(f"[ROI {roi}] RAW v: min={np.min(v):.6g}, max={np.max(v):.6g}")

r_vect = savgol_filter(r, 15, 3)
v_vect = savgol_filter(v, 15, 3)
data_rv = pd.DataFrame({"r": r_vect, "v": v_vect})

sns.set_theme(style="darkgrid")
sns.lineplot(data=data_rv, x="r", y="v", sort=False, lw=2, color="blue", ax=ax2)
ax2.set_xlabel("Firing Rate", fontsize=12)
ax2.set_ylabel("Membrane Potential", fontsize=12)
ax2.set_title("Regional Activity Trajectory Plot (ROI 6)")

ax2.set_xlim((r_min_found - r_pad, 1.8 + r_pad))
ax2.set_ylim((-2.5 - v_pad, 2 + v_pad))

ax2.set_box_aspect(1)



# 3) FCD matrix
ax3 = fig.add_subplot(gs[1, 1])

FCD, _, _ = analysis.compute_fcd(bold_d[:, 0, :, 0], win_len=5)

im = ax3.imshow(FCD, interpolation="nearest")
ax3.set_title("FCD Plot")
ax3.set_xlabel("Timepoint")
ax3.set_ylabel("Timepoint")
ax3.grid(False)

ax3.set_box_aspect(1)
ax3.set_aspect("equal")

cax = inset_axes(ax3, width="4%", height="85%", loc="right", borderpad=1)
plt.colorbar(im, cax=cax, label="Correlation")



# 4) text summary
ax4 = fig.add_subplot(gs[2, :])
ax4.axis("off")
fcd_var = np.var(np.triu(FCD, k=5))
summary_text = (
    f"FCD Variance: {fcd_var:.6e}\n"
)
ax4.text(
    0.5, 0.5, summary_text,
    fontsize=12, ha="center", va="center",
    bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5)
)



# Save img
fig.savefig(
    f"Module-2_output_images/EXPLORATORY_G-{G}_noise-{nsigma}_dt-{dt}_simlen{sim_len}_output-figs.png",
    dpi=60, bbox_inches="tight"
)
plt.show()


### For reference, this is our **baseline** (G=1.99, noise=0.045, dt=0.01. sim_length=10e3):

![baseline_image](Module-2_output_images/BACKUP_BASELINE_G-1.99_noise-0.045_dt-0.01_simlen10000.0_output-figs.png)


<br>

----


####  Findings & Practical Considerations


What does modifying these parameters and simulation settings do?

- G
- noise
- dt
- simulation length

<br>

How do they effect:

- regional activity
- functional network dynamics
- code runtime
- errors
